# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Python-Futures" data-toc-modified-id="Python-Futures-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python Futures</a></div><div class="lev2 toc-item"><a href="#ThreadPoolExecutor" data-toc-modified-id="ThreadPoolExecutor-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>ThreadPoolExecutor</a></div>

# Python Futures

***

The concurrent.futures module provides a high-level interface for asynchronously executing callables.

The asynchronous execution can be performed with threads, using ThreadPoolExecutor, or separate processes, 
using ProcessPoolExecutor. Both implement the same interface, which is defined by the abstract Executor class.

https://docs.python.org/3/library/concurrent.futures.html


## ThreadPoolExecutor
---

ThreadPoolExecutor is an Executor subclass that uses a pool of threads to execute calls asynchronously.

Deadlocks can occur when the callable associated with a Future waits on the results of another Future. For example:

In [5]:
import concurrent.futures
import urllib.request

URLS = ['http://www.foxnews.com/',
        'http://www.cnn.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://some-made-up-domain.com/']

# Retrieve a single page and report the URL and contents
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))

'http://www.foxnews.com/' page is 71014 bytes
'http://www.cnn.com/' page is 137387 bytes
'http://europe.wsj.com/' page is 949327 bytes
'http://www.bbc.co.uk/' page is 230981 bytes
'http://some-made-up-domain.com/' generated an exception: <urlopen error [Errno -2] Name or service not known>


In [4]:
import concurrent.futures
import math

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

def is_prime(n):
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))

